# Analysis

In [ ]:
%reset -f

# Genutzte Umgebung

In [ ]:
import pandas as pd
from logger import Logger
from test_executer import TestExecutor
import extract
import sys
import numpy as np
import hashlib

logger = Logger()
testExecutor = TestExecutor(logger)

dependencies = [
    ('system', sys.version_info),
    ('pandas', pd.__version__),
    ('numpy', np.__version__),
]
for dependency in dependencies:
    logger.log(f"{dependency[0]} is installed with version {dependency[1]}")

## Setup der Daten

Zuerst laden wir die benötigten Daten herunter und initialisieren die genutzten Python Objekte.

In [ ]:
tables = [
    "careplans",
    "conditions",
    "observations",
    "patients",
]

files = [
    "data/others/",
    "data/asthma/",
    "data/gallstones/",
    "data/hypertension/",
]

md5Hashes = {
    "data/others/careplans.csv": "365403e27541792755361bc0f6125506",
    "data/others/conditions.csv": "ce0034e9ed9185b7d4c408ee9916de18",
    "data/others/observations.csv": "b9e3bf1b033dc4af7f7ade78a48a50a4",
    "data/others/patients.csv": "530570c8e30b77a822b37e927d1486b2",
    "data/asthma/careplans.csv": "3c4aff1d0d576de6624c3726ec2dd544",
    "data/asthma/conditions.csv": "e7965095ec41ef88498540341c79c49e",
    "data/asthma/observations.csv": "1b8583de62d4d9e80c224005d74dd736",
    "data/asthma/patients.csv": "b139ef00c850308c3d3f8e7fa0f97724",
    "data/gallstones/careplans.csv": "35399fb01b2771b770c2f9f312e62dc2",
    "data/gallstones/conditions.csv": "8a19bf13191cf074c64534c2fa01f15c",
    "data/gallstones/observations.csv": "9d3807dc05cd7b4ccc3f0ee7b4f7b55e",
    "data/gallstones/patients.csv": "3766f46941ee2155e0d1ed6e749e8ba7",
    "data/hypertension/careplans.csv": "ddf053c4e56f24c4ce28e6d57edfd8b1",
    "data/hypertension/conditions.csv": "8310cdc07924b48e07aa841f9075b488",
    "data/hypertension/observations.csv": "f7564c732eebe9ace17a46e50b3cc857",
    "data/hypertension/patients.csv": "2ebdf6b168e9c968ffa949463cd074e7",
}

In [ ]:
!mkdir -p data/allergy

In [ ]:
from urllib.request import urlopen
import os

def ensure_file_has_been_downloaded(filename):
    full_filename = "../" + filename

    url = "https://raw.githubusercontent.com/Fuenfgeld/DMA2022DataProjectC/main/" + filename
    if os.path.isfile(full_filename):
        logger.log("File {} already exists, skipping download".format(filename))
    else:
        logger.log("Downloading {}".format(filename))
        download_file(url, full_filename)

def download_file(url, filename):
    with open(filename, 'wb') as out_file:
        with urlopen(url) as file:
            out_file.write(file.read())

if not os.path.isfile("extract.py"):
    download_file(
        "https://raw.githubusercontent.com/Fuenfgeld/DMA2022DataProjectC/main/src/extract.py",
        "extract.py"
    )

dataChanged = False
for file in files:
    for table in tables:
        filename = file+table+".csv"
        ensure_file_has_been_downloaded(filename)

        with open("../" + filename) as fileHandle:
            fileContent = fileHandle.read()
            fileHandle.close()

        md5Hash = hashlib.md5(fileContent.encode()).hexdigest()
        if md5Hashes[filename] != md5Hash:
            dataChanged = True
    
if dataChanged:
    logger.log("❌ Data set changed")
else:
    logger.log("✅ Using original data set")

## Mit Datenbank verbinden

In [ ]:
databaseFile = "data.sqlite"

logger.startTimeMeasurement('open-db', 'Connected to db and created tables')
connection = extract.connect_to_db(logger, databaseFile)  # create table patients, observations, conditions, careplans
logger.endTimeMeasurement('open-db')

In [ ]:
def test_sqliteConnection(_logger):
    cursor = connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tablesInDb = list(map(lambda tableResult: tableResult[0], cursor.fetchall()))
    tablesInDb.sort()

    for table in tables:
        if not(table in tablesInDb):
            raise Exception('Table not found:', table)

testExecutor.execute('Test connection to database', test_sqliteConnection)

## Daten in Datenbank laden

Lade der verwendete Daten in die Datenbank:

-   careplans
-   conditions
-   observations
-   patients

In [ ]:
logger.startTimeMeasurement('load-data', 'Loading data into db')
for file in files:
    for table in tables:
        extract.insert_values_to_table(logger, connection.cursor(), table, "../"+ file + table + ".csv")  # TODO: insert ALL values in the right tables 
        connection.commit()

logger.endTimeMeasurement('load-data')

# Annonymisierung

In [ ]:
import pandas as pd
import random
import hashlib

patientDf = pd.read_sql_query('SELECT * FROM patients;', connection)
careplansDf = pd.read_sql_query('SELECT * FROM careplans;', connection)
conditionsDf = pd.read_sql_query('SELECT * FROM conditions;', connection)
observationsDf = pd.read_sql_query('SELECT * FROM observations;', connection)

patientIds = [*patientDf.Id, *careplansDf.PATIENT, *conditionsDf.PATIENT, *observationsDf.PATIENT]

def test_sanityCheckCombiningIds(_logger): 
    expectedLen = len(patientDf) + len(careplansDf) + len(conditionsDf) + len(observationsDf)
    actualLen = len(patientIds)
    if actualLen != expectedLen:
        raise Exception('Not all patient ids were concatenated')

testExecutor.execute('Sanity check: extracting all ids worked', test_sanityCheckCombiningIds)

# Converts list to a set with only unique values
uniqueIds = set(patientIds)
logger.log(f"{len(uniqueIds)} unique patient ids found")
if len(uniqueIds) >= len(patientDf.Id):
    logger.log(f"⚠️ The dataset contains {len(uniqueIds)} unique patientIds but only {len(patientDf.Id)} patients.")

annonymizedIds = {}
for id in uniqueIds:
    # Use uppercase here so it is easy to see if annonymized ids are used.
    annonymizedIds[id] = hashlib.sha256(f"{id}={random.random()}".encode()).hexdigest().upper()

def test_sanityEnsureAllIdsAreAnnonymized(_logger): 
    for id in patientIds:
        if id in annonymizedIds:
            raise Exception('A origin id still exists in anonnymized id list')

testExecutor.execute('Sanity check: no origin ids exist anymore', test_sanityCheckCombiningIds)

logger.startTimeMeasurement('annonymizedPatients', 'Writing annonymized patients')
patientDf = patientDf.replace({"Id": annonymizedIds})
patientDf.to_sql(name="anonnymized_patients", con=connection, if_exists='replace')
logger.endTimeMeasurement('annonymizedPatients')

logger.startTimeMeasurement('annonymizedCareplans', 'Writing annonymized careplans')
careplansDf = careplansDf.replace({"PATIENT": annonymizedIds})
careplansDf.to_sql(name="anonnymized_careplans", con=connection, if_exists='replace')
logger.endTimeMeasurement('annonymizedCareplans')

logger.startTimeMeasurement('annonymizedConditions', 'Writing annonymized conditions')
conditionsDf = conditionsDf.replace({"PATIENT": annonymizedIds})
conditionsDf.to_sql(name="anonnymized_conditions", con=connection, if_exists='replace')
logger.endTimeMeasurement('annonymizedConditions')

logger.startTimeMeasurement('annonymizedObservations', 'Writing annonymized Observations')
observationsDf = observationsDf.replace({"PATIENT": annonymizedIds})
observationsDf.to_sql(name="anonnymized_observations", con=connection, if_exists='replace')
logger.endTimeMeasurement('annonymizedObservations')

## Messung der Datenfehler

Für unsere Forschungsfrage sind nur alle Daten mit gemessenen BMI relevant. Wurde dieser nicht vermessen oder eingetragen können die Daten für die Forschungsfrage nicht verwendet werden und sind somit unbrauchbar.

### NULL-Values

Die Rohdaten werden zuvor auf die Anzahl an NULL-Values überprüft. Weisen mehr als **ein drittel der Daten**   Lücken in der Codierung auf, wird ein Fehler in der Verfassung angenommen und die Daten müssen manuell Überprüft werden.

In [ ]:
null_counter = 0
num_of_elements = 0
for table in tables:
    querie = f"SELECT * from {table};"
    df = pd.read_sql_query(querie,connection)
    result_string = str(df.isna().sum()).replace("\n"," NULL-Values in Column ")
    logger.log(f"Found {result_string} null-values in {table}.")
    null_counter = df.isna().sum().sum() + null_counter
    num_of_elements = num_of_elements + df.size
perc_null_val = round(null_counter / num_of_elements,3)

if perc_null_val > 0.33:
    logger.log(f"Found {perc_null_val} null-values.",type='Warning')
else:
    logger.log(f"Found {perc_null_val} null-values.")

### Prüfung auf Duplikate

Duplikate verfälschen die  Ergebnisse des Anlyseteil durch Steigerung der Grundgesamheit mit gleichen Werten. Somit müssen die Daten auf Duplikate in den einzelnen Files überprüft werden, um gleiche Messungen zu finden und gegebenfalls im ETL-Process zu entfernen.


In [ ]:
num_of_duplicates = 0
num_of_elements = 0
for table in tables:
    querie = f"SELECT * from {table};"
    df = pd.read_sql_query(querie,connection)
    duplicates = df.groupby(df.columns.tolist()).size().reset_index().\
    rename(columns={0:'records'})
    curr_num_duplicate = (duplicates.records -1).sum() 
    num_of_duplicates = num_of_duplicates + curr_num_duplicate
    logger.log(f"Found {curr_num_duplicate} duplicate-values in {table}.")
    num_of_elements = num_of_elements + df.size
perc_duplicates = round(num_of_duplicates / num_of_elements,3)
logger.log(f"Found {perc_duplicates} duplicate-values.")

### Prozentuales Anzahl von Gewichts und BMI Werten für Patieten

Für unsere Forschungsfrage sind BMI-Werte relevant und müssen für den Patienten mindestes einmal codiert worden sein. Um die Forschungsfrage mit den zur vorliegenden Daten zu beanworten, sollten auch hier mindestens **ein drittel der Daten** mit einen BMI Codiert worden sein.


In [ ]:
all_patients_query = """
SELECT COUNT(id) FROM patients;"""
count_bmi_query = """
SELECT COUNT(distinct id) FROM patients JOIN observations on patients.id == observations.patient WHERE observations.Code = '59576-9'"""

count_all_bmi_query = f"""
SELECT COUNT(patient) FROM observations WHERE observations.Code = '59576-9'"""

patient_all_count = connection.execute(all_patients_query).fetchall()[0][0]
patient_bmi_count = connection.execute(count_bmi_query).fetchall()[0][0]
bmi_count = connection.execute(count_all_bmi_query).fetchall()[0][0]
ratio = round(patient_bmi_count/patient_all_count, 3) 

logger.log(f"Total num of patients {patient_all_count}.")

if ratio > 0.33:
    logger.log(f"Found {patient_bmi_count} patients ({round(ratio*100,3)}%) with {bmi_count} BMI-values.")
else:
    logger.log(f"Found {patient_bmi_count} patients ({round(ratio*100,3)}%) with {bmi_count} BMI-values.")


# Star Schema

In [ ]:
cursor = connection.cursor()

#### Erstellen Dimensionstabellen

Tabelle patients_

In [ ]:
# table patients_
cursor.execute('''DROP TABLE IF EXISTS patients_;''')
cursor.execute('''
        CREATE TABLE patients_ ( 
        ID STRING PRIMARY KEY UNIQUE,
        RACE STRING, 
        ETHNICITY STRING,
        GENDER STRING,
        AGE INT64
        );''')

In [ ]:
# create df patients
cursor.execute('''SELECT ID, BIRTHDATE, DEATHDATE, RACE, ETHNICITY, GENDER FROM PATIENTS;''')
df_patients = pd.DataFrame(cursor.fetchall(), columns=['ID', 'BIRTHDATE', 'DEATHDATE', 'RACE', 'ETHNICITY', 'GENDER'])

In [ ]:
# convert to date
df_patients["DEATHDATE"] = pd.to_datetime(df_patients["DEATHDATE"])
df_patients["BIRTHDATE"] = pd.to_datetime(df_patients["BIRTHDATE"])
# fill null values withh todays date
df_patients['DEATHDATE'] = df_patients.DEATHDATE.fillna(pd.to_datetime("today"))
# calculate age
df_patients["AGE"] = df_patients.DEATHDATE.dt.year - df_patients.BIRTHDATE.dt.year
# drop unnecessary variables
df_patients = df_patients.drop(['BIRTHDATE', 'DEATHDATE'], axis=1)

In [ ]:
df_patients.info()

In [ ]:
print("Number of Duplicated Rows", df_patients.duplicated(df_patients.columns).sum())

In [ ]:
df_patients.to_sql('df_patients', connection, if_exists='replace', index=False)
cursor.execute('INSERT INTO patients_ (ID, RACE, ETHNICITY, GENDER, AGE) SELECT ID, RACE, ETHNICITY, GENDER, RACE FROM df_patients;')
cursor.execute('''DROP TABLE IF EXISTS df_patients;''')

print(pd.read_sql_query("PRAGMA table_info(patients_)", connection))

Tabelle observations_

In [ ]:
# table observations_
cursor.execute('''DROP TABLE IF EXISTS observations_;''')
cursor.execute('''
        CREATE TABLE observations_ ( 
        CODE STRING PRIMARY KEY UNIQUE,
        DESCRIPTION STRING,
        UNITS STRING, 
        TYPE STRING
        );''')

# create df observation
cursor.execute('''SELECT CODE, DESCRIPTION, UNITS, TYPE FROM OBSERVATIONS;''')
df_observations = pd.DataFrame(cursor.fetchall(), columns=['CODE','DESCRIPTION', 'UNITS', 'TYPE'])

df_observations = df_observations.drop_duplicates(subset='CODE')

# transform dt in table
df_observations.to_sql('df_observations', connection, if_exists='replace', index=False)

cursor.execute('''INSERT INTO observations_ (CODE, DESCRIPTION, UNITS, TYPE) SELECT CODE, DESCRIPTION, UNITS, TYPE FROM df_observations;''')

cursor.execute('''DROP TABLE IF EXISTS df_observations;''')
print(pd.read_sql_query("PRAGMA table_info(observations_)", connection))
#print(pd.read_sql_query("SELECT * FROM observations_", connection))

Tabelle careplans_code

In [ ]:
# table careplans_code
cursor.execute('''DROP TABLE IF EXISTS careplans_code;''')
cursor.execute('''
        CREATE TABLE careplans_code ( 
        CODE STRING PRIMARY KEY UNIQUE,
        DESCRIPTION STRING
        );''')

# create df careplans_code
cursor.execute('''SELECT CODE, DESCRIPTION FROM CAREPLANS;''')
df_careplans_code = pd.DataFrame(cursor.fetchall(), columns=['CODE','DESCRIPTION'])

df_careplans_code = df_careplans_code.drop_duplicates(subset='CODE')

# transform dt in table
df_careplans_code.to_sql('df_careplans_code', connection, if_exists='replace', index=False)

cursor.execute('''INSERT INTO careplans_code (CODE, DESCRIPTION) SELECT CODE, DESCRIPTION FROM df_careplans_code;''')

cursor.execute('''DROP TABLE IF EXISTS df_careplans_code;''')
print(pd.read_sql_query("PRAGMA table_info(careplans_code)", connection))
#print(pd.read_sql_query("SELECT * FROM careplans_code", connection))

Tabelle careplans_reasoncode

In [ ]:
# table careplans_reasoncode
cursor.execute('''DROP TABLE IF EXISTS careplans_reasoncode;''')
cursor.execute('''
        CREATE TABLE careplans_reasoncode ( 
        REASONCODE STRING PRIMARY KEY UNIQUE,
        REASONDESCRIPTION STRING
        );''')

# create df careplans_code
cursor.execute('''SELECT REASONCODE, REASONDESCRIPTION FROM CAREPLANS;''')
df_careplans_reasoncode = pd.DataFrame(cursor.fetchall(), columns=['REASONCODE','REASONDESCRIPTION'])

df_careplans_reasoncode = df_careplans_reasoncode.drop_duplicates(subset='REASONCODE')

# transform dt in table
df_careplans_reasoncode.to_sql('df_careplans_reasoncode', connection, if_exists='replace', index=False)
   
cursor.execute('INSERT INTO careplans_reasoncode (REASONCODE, REASONDESCRIPTION) SELECT REASONCODE, REASONDESCRIPTION FROM df_careplans_reasoncode;')

cursor.execute('''DROP TABLE IF EXISTS df_careplans_reasoncode;''')
print(pd.read_sql_query("PRAGMA table_info(careplans_reasoncode)", connection))
#print(pd.read_sql_query("SELECT * FROM careplans_reasoncode", connection))

Tabelle conditions_

In [ ]:
# table conditions
cursor.execute('''DROP TABLE IF EXISTS conditions_;''')
cursor.execute('''
        CREATE TABLE conditions_ ( 
        CODE STRING PRIMARY KEY UNIQUE,
        DESCRIPTION STRING
        );''')

# create df conditions
cursor.execute('''SELECT CODE, DESCRIPTION FROM CONDITIONS;''')
df_conditions = pd.DataFrame(cursor.fetchall(), columns=['CODE','DESCRIPTION'])

df_conditions = df_conditions.drop_duplicates(subset='CODE')

# transform dt in table
df_conditions.to_sql('df_conditions', connection, if_exists='replace', index=False)

cursor.execute('INSERT INTO conditions_ (CODE, DESCRIPTION) SELECT CODE, DESCRIPTION FROM df_conditions;')

cursor.execute('''DROP TABLE IF EXISTS df_conditions;''')
print(pd.read_sql_query("PRAGMA table_info(conditions_)", connection))
#print(pd.read_sql_query("SELECT * FROM conditions_", connection))

Datumstabelle

In [ ]:
from datetime import datetime
def create_date_table(start='1900-01-01', end=datetime.today().strftime('%Y-%m-%d')):
    
    df_date = pd.DataFrame({"Date": pd.date_range(start, end)})

    days_names = {
        i: name
        for i, name
        in enumerate(['Monday', 'Tuesday', 'Wednesday',
                      'Thursday', 'Friday', 'Saturday', 
                      'Sunday'])
    }
   
    df_date["Day"] = df_date.Date.dt.dayofweek.map(days_names.get)
    df_date["Week"] = df_date.Date.dt.weekofyear
    df_date["Quarter"] = df_date.Date.dt.quarter
    df_date["Year"] = df_date.Date.dt.year
    df_date["Year_half"] = (df_date.Quarter + 1) // 2
    
    return df_date

In [ ]:
# table date_table
cursor.execute('''DROP TABLE IF EXISTS date_table;''')
cursor.execute('''
        CREATE TABLE date_table ( 
        DATE DATE PRIMARY KEY UNIQUE,
        DAY STRING,
        WEEK INT16,
        QUARTER INT16,
        YEAR INT16,
        YEAR_HALF INT16
        );''')

In [ ]:
df_date = create_date_table()

# transform dt in table
df_date.to_sql('df_date', connection, if_exists='replace', index=False)

cursor.execute('INSERT INTO date_table (DATE, DAY, WEEK, QUARTER, YEAR, YEAR_HALF) SELECT DATE, DAY, WEEK, QUARTER, YEAR, YEAR_HALF FROM df_date;')

cursor.execute('''DROP TABLE IF EXISTS df_date;''')
print(pd.read_sql_query("PRAGMA table_info(date_table)", connection))

#### Erstelle Faktentabelle

In [ ]:
cursor.execute('''DROP TABLE IF EXISTS fact_table;''')
cursor.execute('''
        CREATE TABLE fact_table ( 
        PATIENT_ID STRING,
        OBSERVATION_CODE STRING,
        VALUE STRING,
        DATE DATE,
        CONDITIONS_CODE STRING,
        ENDDATE DATE,
        CAREPLANS_CODE STRING,
        CAREPLANS_REASONCODE STRING,
        FOREIGN KEY (PATIENT_ID)
            REFERENCES patients_(ID),
        FOREIGN KEY (OBSERVATION_CODE)
            REFERENCES observations_(CODE),
        FOREIGN KEY (CONDITIONS_CODE)
            REFERENCES conditions_(CODE),
        FOREIGN KEY (CAREPLANS_CODE)
            REFERENCES careplans_code(CODE),
        FOREIGN KEY (CAREPLANS_REASONCODE)
            REFERENCES careplans_reasoncode(REASONCODE)
        );''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, OBSERVATION_CODE, VALUE, DATE) 
                    SELECT PATIENT, CODE, VALUE, DATE 
                    FROM OBSERVATIONS
                    ;''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, CAREPLANS_CODE, DATE, ENDDATE) 
                    SELECT PATIENT, CODE, START, STOP 
                    FROM CAREPLANS
                    ;''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, CAREPLANS_REASONCODE, DATE, ENDDATE) 
                    SELECT PATIENT, REASONCODE, START, STOP 
                    FROM CAREPLANS
                    ;''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, CONDITIONS_CODE, DATE, ENDDATE) 
                    SELECT PATIENT, CODE, START, STOP 
                    FROM CONDITIONS
                    ;''')

In [ ]:
print(pd.read_sql_query("PRAGMA foreign_key_list(fact_table)", connection))

In [ ]:
cursor.execute('SELECT name FROM sqlite_master where type="table"')
print(cursor.fetchall())

## Analysis

Show the the percentile of a patient's body mass index indicates the relative position of the patient's BMI number among the given population. For children and teens the interpretation of BMI is both age- and sex-specific.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

df_bmi_perc = pd.read_sql("SELECT * FROM fact_table WHERE observation_code = '59576-9'",connection)
df_bmi_perc['VALUE'].hist()

In [ ]:
df_bmi = pd.read_sql("SELECT * FROM fact_table WHERE observation_code ='39156-5'",connection)
plt.hist(df_bmi['VALUE'])

In [ ]:
df_height_weight = pd.read_sql("SELECT distinct fact_table.patient_id, fact_table.value,weight.value FROM fact_table left JOIN (SELECT distinct patient_ID,value FROM fact_table WHERE observation_code = '29463-7') AS weight on weight.patient_ID==fact_table.patient_id WHERE observation_code = '8302-2' ",connection)


In [ ]:
# TODO fix 
df_height_weight = pd.read_sql("SELECT distinct fact_table.patient_id, fact_table.value,weight.value FROM fact_table left JOIN (SELECT distinct patient_ID,value FROM fact_table WHERE observation_code = '29463-7') AS weight on weight.patient_ID==fact_table.patient_id WHERE observation_code = '8302-2' ",connection)
df_height_weight = df_height_weight.rename({'VALUE': 'height', 'value': 'weight'}, axis=1)
plt.scatter(df_height_weight.height,df_height_weight.weight)

In [ ]:
df_con = pd.read_sql("SELECT conditions.patient,conditions.code,conditions.description,observations.value\
 FROM conditions join observations on observations.patient==conditions.patient WHERE observations.Code = '39156-5' AND conditions.description NOT LIKE '%finding%'",connection)
plt.pie(df_con['DESCRIPTION'].value_counts()[:10],labels=df_con['DESCRIPTION'].value_counts()[:10].index)
plt.plot()

### Verteilung Oberservation/Conditions

Verschiedener Beobachtungen zu den Top 5 Krankheitesaufkommen in unseren Datensatz.



In [ ]:
top_five = pd.DataFrame(df_con['DESCRIPTION'].value_counts()[:5]).reset_index()
top_five = top_five.rename({'index': 'DESCRIPTION', 'DESCRIPTION': 'COUNT'}, axis=1)
top_five['CODE'] = df_con['CODE'].value_counts()[:5].index
top_five

- 8302-2,Body Height
- 29463-7,Body Weight
- 39156-5,Body Mass Index
- 8462-4,Diastolic Blood Pressure
- 8480-6,Systolic Blood Pressure
- 8867-4,Heart rate

In [ ]:
disease_labels = ['Viral sinusitis (disorder)','Acute viral pharyngitis (disorder)','Hypertension','Acute bronchitis (disorder)','Prediabetes']
disease_codes = [444814009,195662009,59621000,10509002,15777000]
observation_lables = ['Body Height','Body Weight','Body Mass Index','Diastolic Blood Pressure','Systolic Blood Pressure','Heart rate']
observation_codes = ['8302-2','29463-7','39156-5','8462-4','8480-6','8867-4']
df_lists = list()
for dis_code,dis_lable in zip(disease_codes,disease_labels):
    df = pd.DataFrame()
    for ob_code,ob_lable in zip(observation_codes,observation_lables):
        sql_pattern = f"""
        SELECT observations.date,conditions.patient,conditions.code,observations.value
            FROM conditions join observations on observations.patient==conditions.patient
            WHERE observations.Code = '{ob_code}' AND conditions.code = '{dis_code}'"""
        df[ob_lable] = pd.read_sql(sql_pattern,connection)['VALUE']
    df['Disease'] = dis_lable
    df_lists.append(df)
df_data = pd.concat(df_lists).reset_index(drop=True)
df_data.head()

In [ ]:
df_data['class'] = ''
df_data.loc[df_data['Body Mass Index'].values<18,'class'] = 'underweight'
df_data.loc[df_data['Body Mass Index'].values>=18,'class'] = 'normal'
df_data.loc[df_data['Body Mass Index'].values>=25,'class'] = 'overweight'
df_data.loc[df_data['Body Mass Index'].values>=30,'class'] = 'obese'
df_data.head()

In [ ]:
sns.set(style="darkgrid")
ax = sns.boxplot(x="Body Mass Index", y="Disease", data=df_data)
plt.axvline(x=25, c='red',ls=':')
plt.show()

In [ ]:
ax = sns.ecdfplot(data=df_data, x="Body Mass Index",hue='Disease')
plt.axvline(x=25, c='red',ls=':')
plt.show()

In [ ]:
df_hypertension = df_data.where(df_data['Disease']== 'Hypertension')
sns.barplot(x=df_hypertension['class'].value_counts().index,y=df_hypertension['class'].value_counts().values)
plt.title("Hypertension")
plt.show()

### [Regression](https://scikit-learn.org/stable/auto_examples/cross_decomposition/plot_pcr_vs_pls.html#sphx-glr-auto-examples-cross-decomposition-plot-pcr-vs-pls-py)

In [ ]:
from sklearn.linear_model import LogisticRegression

df_reg = df_data.dropna()
X, y = df_reg.drop(['Disease','class'],axis=1)['Body Mass Index'].values.reshape((-1, 1)), df_reg['Disease']
model = LogisticRegression(multi_class='multinomial', solver='lbfgs').fit(X,y)

model.predict(np.array([25,]).reshape(-1, 1))

### [Decision tree](https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html#sphx-glr-auto-examples-tree-plot-unveil-tree-structure-py)

In [ ]:
from sklearn import tree
df_tree = df_data.dropna()
X, y = df_tree.drop(['Disease','class'],axis=1), df_tree['Disease']
clf = tree.DecisionTreeClassifier(max_depth=5,max_features='sqrt',random_state=42)
clf = clf.fit(X, y)
print(X.columns)

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.image as mpimg

plt.figure()

# need to munch time to run in CI
plot_tree(clf, filled=True)
plt.savefig('../images/decision_tree.png',dpi=800)
plt.title("Decision tree")

plt.show()

## Aufräumen & Logs speichern

In [ ]:
connection.close()
logger.logTimings()
logger.writeToFile("../artefacts-for-release/analysis-log.json")